In [15]:
#coding=utf-8
import csv

In [19]:
csv_reader = csv.reader(open('data/match_station.csv', encoding='utf-8'))
Dict = {}
for row in csv_reader:
    Dict[row[0]] = row[1]

{'aotizhongxin_aq': 'beijing_grid_304',
 'beibuxinqu_aq': 'beijing_grid_263',
 'dongsi_aq': 'beijing_grid_303',
 'fengtaihuayuan_aq': 'beijing_grid_282',
 'guanyuan_aq': 'beijing_grid_282',
 'gucheng_aq': 'beijing_grid_261',
 'nongzhanguan_aq': 'beijing_grid_325',
 'tiantan_aq': 'beijing_grid_304',
 'wanliu_aq': 'beijing_grid_283',
 'wanshouxigong_aq': 'beijing_grid_303',
 'yungang_aq': 'beijing_grid_239',
 'zhiwuyuan_aq': 'beijing_grid_262'}

In [41]:
#找对应项，构建列表
csv_reader1 = csv.reader(open('data/beijing_17_18_aq.csv', encoding='utf-8'))
station_list = []
for i in Dict:
    station_list.append(Dict[i])
station_list = list(set(station_list))
station_list = sorted(station_list)

for s0 in station_list:
    for row in csv_reader1:
        row
        #print(row[0])
        #if s0 == row[0]:
         
            #print(row[0])
#for row in csv_reader:
 #   if row[0] 
        
#填写文件
#out = open('data/beijing_17_18_aq.csv', 'w', newline='')
#csv_writer = csv.writer(out, dialect='excel')
